In [1]:
import gurobipy as gp
from gurobipy import GRB
from gurobipy import *  # imports everything from gurobipy without alias
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random

In [2]:
model = gp.Model("MILP")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-10-27


In [3]:
# sets
tasks = range(30)  # Set of tasks
mated_stations = range(10)  # Set of mated stations
sides = ['left-side', 'right-side', 'beneath-side', 'above-side']
agents = range(20)  # Set of agents
agents_human = range(10)  # Index of humans in the subset of agents
agents_robot = range(10,20)  # Index of robots in the subset of agents
product_models = range(3)  # Set of product models

J, I, K = len(tasks), len(tasks), len(tasks)
M, S = len(mated_stations), len(mated_stations)
N = len(product_models)
H, L = len(sides), len(sides)
A = len(agents)

# Extention #
CO = [random.uniform(30, 50) for a in range(len(agents_human))] # Cost of human wage
CRP = [random.uniform(1000, 10000) for a in range(len(agents_robot))] # Cost of purcahsing robot
CRM = [random.uniform(0, 5)+random.random() for a in range(len(agents_robot))] # Cost of maintaining robot
MB = 50000 # Maximum budget buying robots 
#############

# Specific Task Sets
ls = {task - 1 for task in [1, 2, 3, 4]}  # Adjust left-side tasks
rs = {task - 1 for task in [16, 17, 18, 19, 20]}  # Adjust right-side tasks
bs = {task - 1 for task in [5, 6, 7, 8, 9, 10]}  # Adjust both-side tasks
us = {task - 1 for task in [21, 22, 23, 24, 25]}  # Adjust beneath-side tasks
ps = {task - 1 for task in [26, 27, 28, 29, 30]}  # Adjust above-side tasks
to = {task - 1 for task in [2, 5, 7, 13, 18]}  # Tasks only humans can do
tr = {task - 1 for task in [9, 11, 16, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]}  # Tasks only robots can do
tb = {task - 1 for task in [1, 3, 4, 6, 8, 10, 12, 14, 15, 17, 19]}  # Tasks both humans and robots can do

t = {(a, j, n): random.uniform(3, 15) for n in range(N) for j in range(J) for a in range(A)}  # processing time of task j for model n by agent a

# Precedence and Zoning
pi = {(j - 1, i - 1) for j, i in [(1, 2)]}  # Immediate predecessors
sa = {(j - 1, i - 1) for j, i in [
    (2, 8), (2, 9), (4, 11), (5, 10), (5, 30),
    (9, 12), (9, 14), (11, 12), (11, 16),
    (15, 23), (15, 30), (2, 12), (2, 14),
    (4, 12), (4, 16)
]}  # Immediate successors
po = {(j - 1, i - 1) for j, i in [(1, 2)]}  # Positive zoning
ne = {(j - 1, i - 1) for j, i in [(1, 30), (1, 5)]}  # Negative zoning
se = {(j - 1, i - 1) for j, i in [(4, 6)]}  # Synchronous tasks

phi = 10e6
mu1 = 0.3 # The opened mated-station weight
mu2 = 0.6 # The opened station weight

In [4]:
# Preferred sides (directions) for task j
H_j = {}
for j in range(J):
    if j in rs:
        H_j[j] = {0}  # Right-side
    elif j in ls:
        H_j[j] = {1}  # Left-side
    elif j in us:
        H_j[j] = {2}  # Beneath-side
    elif j in ps:
        H_j[j] = {3}  # Above-side
    elif j in bs:
        H_j[j] = {0, 1}  # Left and right sides
    else:
        H_j[j] = set()  # No valid sides (optional)

In [5]:
# Decision variables
x = model.addVars(J, M, H, vtype=GRB.BINARY, name = 'task j is assigned to mated-station m with side h')
z = model.addVars(M, H, vtype=GRB.BINARY, name = 'station m is utilized for a model')
y = model.addVars(J, K, vtype=GRB.BINARY, name = 'task j is assigned earlier than task k in the same station')
q = model.addVars(A, H, M, vtype=GRB.BINARY, name = 'agent a is assigned to side h in mated-station m')
tf = model.addVars(J, N, vtype=GRB.CONTINUOUS, name = 'Finishing time of task j for model n')
rr = model.addVars(J, N, vtype=GRB.CONTINUOUS, name = 'Real time of task j for model n' )
alpha = model.addVars(M, vtype=GRB.BINARY, name = 'just one side of mated-station is utilized')
beta = model.addVars(M, vtype=GRB.BINARY, name = 'two sides of mated-station are utilized')
theta = model.addVars(M, vtype=GRB.BINARY, name = 'three sides of mated-station are utilized')
gamma = model.addVars(M, vtype=GRB.BINARY, name = 'four sides of mated-station are utilized')
v1 = model.addVars(M,  H, vtype=GRB.BINARY, name = 'at least one task to in mated-station m with side h')
v2 = model.addVars(M,  H, vtype=GRB.BINARY, name = 'at least one task tr in mated-station m with side h')
cy = model.addVar(lb=0, name="cycle time non negative")
# Extension #
yr = model.addVars(len(agents_robot), vtype=GRB.BINARY, name="RobotPurchased")  # whether robot[a] is purchased
#############

Objective considering human workers and robot costs</br></br>
$
\sum_{a \in a(O)} CO_{a} \cdot q_{ahm} + 
(\sum_{a \in a(R)} CRP_{a} \cdot yr_{a}+
\sum_{a \in a(R)} CRM_{a} \cdot q_{ahm})
$

In [6]:
# Objectives
# model.setObjective(
#     quicksum((CO[a]*(quicksum(quicksum(q[a,h,m] for m in range(M)) for h in range(H)))) for a in agents_human) +
#     quicksum(CRP[a]*yr[a] for a in range(len(agents_robot))) +
#     quicksum((CRM[a-10]*(quicksum(quicksum(q[a,h,m] for m in range(M)) for h in range(H)))) for a in agents_robot),
#     GRB.MINIMIZE
# ) # single-objective
# 1
model.setObjectiveN(
    mu1 * quicksum((alpha[m] + beta[m] + theta[m] + gamma[m]) for m in range(M)) + mu2 * quicksum(quicksum(z[m,h] for h in range(H)) for m in range(M)),
    index=0,
    priority=0
) # multi-objective1
#2
model.setObjectiveN(
    quicksum((CO[a]*(quicksum(quicksum(q[a,h,m] for m in range(M)) for h in range(H)))) for a in agents_human) +
    quicksum(CRP[a]*yr[a] for a in range(len(agents_robot))) +
    quicksum((CRM[a-10]*(quicksum(quicksum(q[a,h,m] for m in range(M)) for h in range(H)))) for a in agents_robot),
    index=1,
    priority=1 # higer priority
) # multi-objective2

In [7]:
# Constraints
# 3
model.addConstrs((quicksum(quicksum(x[j, m, h] for h in range(H)) for m in range(M)) == 1 for j in range(J)), name="Task_Assignment")
# 4
model.addConstrs((quicksum(x[j, m, 2] for m in range(M)) == 1 for j in us), name="Beneath_Task_Assignment")
# 5
model.addConstrs((quicksum(x[j, m, 3] for m in range(M)) == 1 for j in ps), name="Above_Task_Assignment")
# 6
model.addConstrs((quicksum(quicksum(q[a, h, m] for h in range(H)) for m in range(M)) <= 1 for a in range(A)), name="Agent_Assignment")
# 7
model.addConstrs(quicksum(q[a, h, m] for a in range(A)) == z[m, h] for h in range(H) for m in range(M))
# 8
model.addConstrs(quicksum(q[a, 3, m] for a in agents_human) <= 0 for m in range(M))
# 9
model.addConstrs(tf[j, n] <= cy for j in range(J) for n in range(N))
# 10
model.addConstrs(tf[j, n] >= rr[j, n] for j in range(J) for n in range(N))
# 11
model.addConstrs(
    (rr[j, n]
        == quicksum(quicksum(quicksum(t[a, j, n] * x[j, m, h] * q[a, h, m] for m in range(M)) for a in range(A)) for h in range(H)))
    for j in range(J) for n in range(N)
)
# 12
for i, j in pi:
    model.addConstr(
        (gp.quicksum(s * x[i, s, h] for s in mated_stations for h in range(len(sides)))
        <= gp.quicksum(m * x[j, m, h] for m in mated_stations for h in range(len(sides)))
        ), name="Precedence_Different_Mated-Station"
    ) ### for only one element in pi
# 13
model.addConstrs(
    (tf[j, n] - tf[i, n] + phi * (1 - gp.quicksum(x[i, m, h] for h in range(len(sides)))) + phi * (1 - gp.quicksum(x[j, m, h] for h in range(len(sides))))
        >= rr[j, n] for i, j in pi  # Loop over the precedence pairs
        for m in mated_stations for n in product_models), name="Precedence_Same_Mated-Station"
) ### for only one element in pi

{(0, 1, 0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 4, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 4, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 4, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 5, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 5, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 5, 2): <gurobi.Constr *Awaiting Model Up

In [8]:
# Equation (14): Precedence for tasks with no predecessor or successor relationships
model.addConstrs(
    (
        tf[k, n] - tf[j, n] +
        phi * (1 - x[j, m, h]) + phi * (1 - x[k, m, h]) +
        phi * (1 - y[j, k]) >= rr[k, n]
        for j in tasks for n in product_models for k in tasks
        if j < k and (j, k) not in pi and (j, k) not in sa
        for m in mated_stations for h in range(len(sides))
    ),
    name="Precedence_No_Relationships"
)

# Equation (15): Precedence for tasks with no direct relationships
model.addConstrs(
    (
        tf[j, n] - tf[k, n] +
        phi * (1 - x[j, m, h]) + phi * (1 - x[k, m, h]) +
        phi * y[j, k] >= rr[j, n]
        for j in tasks for n in product_models for k in tasks
        if j < k and (j, k) not in pi and (j, k) not in sa
        for m in mated_stations for h in range(len(sides))
    ),
    name="Precedence_No_Direct"
)

# Equation (16): Ensure task assignments imply station utilization
model.addConstrs(
    (
        gp.quicksum(x[j, m, h] for j in tasks) - phi * z[m, h] <= 0
        for m in mated_stations for h in range(len(sides))
    ),
    name="Station_Utilization"
)

# Equation (17): Calculate the number of sides utilized in a station
model.addConstrs(
    (
        gp.quicksum(z[m, h] for h in range(len(sides))) -
        4 * gamma[m] - 3 * theta[m] - 2 * beta[m] - alpha[m] == 0
        for m in mated_stations
    ),
    name="Sides_Utilized"
)

# Equation (18): Positive zoning constraints
model.addConstrs(
    (
        x[j, m, h] - x[i, m, h] == 0
        for (j, i) in po for m in mated_stations for h in H_j[j] & H_j[i]
    ),
    name="Positive_Zoning"
)

# Equation (19): Negative zoning constraints
model.addConstrs(
    (
        gp.quicksum(x[j, m, h] for h in H_j[j]) +
        gp.quicksum(x[i, m, h] for h in H_j[i]) <= 1
        for (j, i) in ne for m in mated_stations
    ),
    name="Negative_Zoning"
)

# Equation (20): Synchronous task constraints
model.addConstrs(
    (
        x[j, m, l] - x[i, m, h] == 0
        for (j, i) in se for m in mated_stations for h in H_j[i] for l in H_j[j]
        if h != l
    ),
    name="Synchronous_Tasks"
)

# Equation (21): Ensure synchronous tasks complete together
model.addConstrs(
    (
        tf[j, n] - rr[j, n] == tf[i, n] - rr[i, n]
        for (j, i) in se for n in product_models
    ),
    name="Synchronous_Completion"
)

# Equation (22): Ensure humans are assigned to human-exclusive tasks
model.addConstrs(
    (
        gp.quicksum(x[j, m, h] for j in to) <= phi * v1[m, h]
        for m in mated_stations for h in H_j[j]
    ),
    name="Human_Assignment_Upper"
)
model.addConstrs(
    (
        gp.quicksum(x[j, m, h] for j in to) >= v1[m, h]
        for m in mated_stations for h in H_j[j]
    ),
    name="Human_Assignment_Lower"
)

# Equation (23): Ensure robots are assigned to robot-exclusive tasks
model.addConstrs(
    (
        gp.quicksum(x[j, m, h] for j in tr) <= phi * v2[m, h]
        for m in mated_stations for h in H_j[j]
    ),
    name="Robot_Assignment_Upper"
)
model.addConstrs(
    (
        gp.quicksum(x[j, m, h] for j in tr) >= v2[m, h]
        for m in mated_stations for h in H_j[j]
    ),
    name="Robot_Assignment_Lower"
)

# Equation (24): Limit agent assignment to stations with humans
model.addConstrs(
    (
        gp.quicksum(q[a, h, m] for a in agents_robot) <= 1 - v1[m, h]
        for m in mated_stations for h in [0,1,2,3]
    ),
    name="Agent_Human_Assignment"
)

# Equation (25): Limit agent assignment to stations with robots
model.addConstrs(
    (
        gp.quicksum(q[a, h, m] for a in agents_human) <= 1 - v2[m, h]
        for m in mated_stations for h in range(len(sides))
    ),
    name="Agent_Robot_Assignment"
)

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (3, 3): <gurobi.Constr *Awaiting Model Update*>,
 (4, 0): <gurobi.Constr *Awaiting Model Update*>,
 (4, 1): <gurobi.Constr *Awaiting Model Update*>,
 (4, 2): <gurobi.Constr *Awaiting Model Update*>,
 (4, 3): <gurobi.Constr *Awaiting Model Update*>,


$
yr_{a} \geq \sum_{h \in H} \sum_{m \in M} q_{ahm} \quad \forall a \in a(o)
$  ---(26) </br></br>
Constraint (26) reflects the purchasement of robot if a robot agent is assigned to at least one task</br></br>
$
\sum_{a \in a(R)} CRP_{a} \cdot yr_{a} \leq MB
$  ---(27) </br></br>
Constraint (27) restricts that the total robot purchasing cost should not be larger than the maximum budget</br></br>

Eq(27) Reference: </br>
Li et al. (2021). Multi-objective migrating bird optimization algorithm for cost-oriented assembly line balancing problem with collaborative robots. Neural Comput & Applic 33, 8575–8596. 

In [9]:
# Extension #
# Equation (26): Link robot purchasing to task assignment
model.addConstrs(
    (
        yr[a-10] >= gp.quicksum(q[a, h, m] for h in range(len(sides)) for m in mated_stations)
        for a in agents_robot
    ),
    name="Robot_Purchasing_Link"
)

# Equation (27): Limit total robot purchasing cost
model.addConstr(
    (
        gp.quicksum(CRP[a-10] * yr[a-10] for a in agents_robot) <= MB
    ),
    name="Agent_Robot_Budget"
)

<gurobi.Constr *Awaiting Model Update*>

In [10]:
model.update()

In [11]:
model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) Ultra 5 125H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 18 logical processors, using up to 18 threads

Optimize a model with 101105 rows, 3251 columns and 610304 nonzeros
Model fingerprint: 0xa5091c25
Model has 90 quadratic constraints
Variable types: 181 continuous, 3070 integer (3070 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  QMatrix range    [3e+00, 1e+01]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [3e-01, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+07]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 2 objectives ... 
---------------------------------------------------------------------------

Multi-objectives: applying initial presolve ...
---------------------------------------------------------------------------



In [12]:
# Print results
results = {}
if model.status == GRB.OPTIMAL:
    # print(f"Objective (Minimize Total Cost): {model.ObjVal}") # single-objective
    print(f"Objective 1 (Minimize Mated Stations): {model.getObjective(0).getValue()}")
    print(f"Objective 2 (Minimize Total Costs): {model.getObjective(1).getValue()}")
    print("Solution:")
    for j in tasks:
        for m in mated_stations:
            for h in range(len(sides)):
                if x[j, m, h].x > 0.5:  # Check if the variable is active
                    print(f"Task {j+1} is assigned to Station {m+1} on {sides[h]}.")
    print("-"*50)
    for a in agents:
        for h in range(len(sides)):
            for m in mated_stations:
                if q[a, h, m].x > 0.5:  # Check if the variable is active
                    print(f"Agent {a+1} is assigned to Station {m+1} on {sides[h]}.")
    print("-"*50)
    for a in agents_robot:
        if yr[a-10].x > 0:
            print(f"Agent{a+1} (Robot {a-10+1}) is purchased.")

else:
    print("No solution found")

Objective 1 (Minimize Mated Stations): 1.5
Objective 2 (Minimize Total Costs): 1240.0405403930622
Solution:
Task 1 is assigned to Station 10 on above-side.
Task 2 is assigned to Station 10 on above-side.
Task 3 is assigned to Station 10 on above-side.
Task 4 is assigned to Station 10 on beneath-side.
Task 5 is assigned to Station 10 on above-side.
Task 6 is assigned to Station 10 on above-side.
Task 7 is assigned to Station 10 on above-side.
Task 8 is assigned to Station 10 on above-side.
Task 9 is assigned to Station 10 on above-side.
Task 10 is assigned to Station 10 on above-side.
Task 11 is assigned to Station 10 on above-side.
Task 12 is assigned to Station 10 on above-side.
Task 13 is assigned to Station 10 on above-side.
Task 14 is assigned to Station 10 on above-side.
Task 15 is assigned to Station 10 on above-side.
Task 16 is assigned to Station 10 on above-side.
Task 17 is assigned to Station 10 on above-side.
Task 18 is assigned to Station 10 on above-side.
Task 19 is assign